# **Основные этапы работы:**
## Этап 0. Установка и настройка оболочки для работы с языком Python
Загрузка необходимых библиотек для выполнения лаборатрных работ:

In [3]:
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import keras
from keras import models
from keras import layers
from keras.utils import to_categorical
IMG_HEIGHT = IMG_WIDTH = 150
import os, shutil
import pandas as pd
from pathlib import Path
import os.path
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import random
IMG_HEIGHT = IMG_WIDTH = 150

#! pip install -q kaggle

## Этап 1. Построение сверточной нейронной сети для распознавания объектов из базы данных MNIST.

**Целью данного этапа** лабораторной работы является создание сверточного многокатегориального классификатора изображений цифр из набора данных MNIST.

Вариант этапа 1
| № | Количество слоев | Количество карт признаков на слое | Размер ядра свертки | Количество скрытых слоев классификатора (количество нейронов на слое)|
| :-: | :-: | :-: | :-: | :-: |
| 3 | 2 | 64-64 | 3x3 | 1 (512) |

 Пошаговая реализация поставленной цели включает:
    1. Загрузка набора данных MNIST:
Набор данных MNIST — большой (порядка 60 000 тренировочных и 10 000 проверочных объектов, помеченных на принадлежность одному из десяти классов — какая цифра изображена на картинке) набор картинок с рукописными цифрами, часто используемый для тестирования различных алгоритмов распознавания образов. Он содержит черно-белые картинки размера 28x28 пикселей, исходно взятые из набора образцов из бюро переписи населения США, к которым были добавлены тестовые образцы, написанные студентами американских университетов. 

    2. Разделение данных на обучающий и тестовый наборы
Загрузка данных осуществляется из библиотеки Keras. Данные изначально разделены на тренировочные и тестовые в соотношении 6:1. Размер обучающего набора составляет 60000 экземпляров.

In [4]:
from keras.datasets import mnist
(train_images, train_labels),(test_images, test_labels)=mnist.load_data()

<center><img src="2.1.1.png"></center>
<center>Рис. 1. Образцы изображений из базы MNIST</center>

Представленные данные из базы MNIST — изображения — хранятся в трехмерном массиве (60000, 28, 28) типа uint8, значениями в котором являются числа в интервале [0, 255]. Прежде чем использовать их для обучения нейронной сети, мы сначала выполним предварительную обработку данных, нормализуем эти векторы и преобразуем метки классов к векторному представлению. Преобразуем их в форму (60000, 28 х 28) типа float32, которую ожидает получить нейронная сеть, и масштабируем их так, чтобы все значения оказались в интервале [0, 1].

    3. Подготовка данных для передачи в нейронную сеть

In [5]:
train_images = train_images.reshape((60000,28,28,1))
train_images=train_images.astype('float32')/255

test_images = test_images.reshape((10000,28,28,1))
test_images=test_images.astype('float32')/255

Для меток используем прямое кодирование (one-hot encoding). Прямое кодирование широко используется для форматирования категорий и также называется кодированием категорий (categorical encoding):

In [6]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_images.shape)
train_labels.shape

(60000, 28, 28, 1)


(60000, 10)

    4. Выполнить конструирование сети: создание сверточной основы в соответствии с вариантом, добавить к сверточной основе классификатор заданной архитектуры, вид активационной функции на сверточном слое и в скрытых слоях классификатора выбрать relu:

In [7]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Важно отметить, что данная сеть принимает на входе тензоры с формой (высота_ изображения, ширина_изображения, каналы) (не включая измерение, определяющее пакеты). В данном случае мы настроили сеть на обработку входов с размерами (28, 28, 1), соответствующими формату изображений в наборе MNIST, передав аргумент input_shape=(28, 28, 1) в первый слой.

С помощью функции _Sequential()_ создаём новую модель. Модель будет состоять из разных видов слоев, сверточных, субдискретизирующих и полносвязных.

<center><img src="n-cnn.webp"></center>
<center>Рис. 2. Визуализация архитектуры CNN (сверточной нейронной сети)</center>

Сверточная сеть состоит из слоев Conv2D и MaxPooling2D. Все слои, Conv2D и MaxPooling2D, выводят трехмерный тензор с формой (высота, ширина, каналы). Измерения ширины и высоты сжимаются с ростом глубины сети. Количество каналов управляется первым аргументом, передаваемым в слои Conv2D (64).

Основное отличие полносвязного слоя от сверточного заключается в следующем: слои Dense изучают глобальные шаблоны в пространстве входных признаков (например, в случае с цифрами из набора MNIST это шаблоны, вовлекающие все пиксели), тогда как сверточные слои изучают локальные шаблоны: в случае с изображениями — шаблоны в небольших двумерных окнах во входных данных. Эта выходная карта признаков также является 

У светрочных нейронных сетей есть 2 важных свойства:
1. Шаблоны, которые они изучают, являются инвариантными в отношении переноса.
2. Они могут изучать пространственные иерархии шаблонов.

Свертка применяется к трехмерным тензорам, называемым картами признаков, с двумя пространственными осями (высота и ширина), а также с осью глубины, для черно-белых изображений она имеет размерность 1. Операция свертывания извлекает шаблоны из своей входной карты признаков и применяет одинаковые преобразования ко всем шаблонам,
производя выходную карту признаков трехмерным тензором.

Свертки определяются двумя ключевыми параметрами:
1. Размер шаблонов, извлекаемых из входных данных, согласно варианту (3 × 3).
2. Глубина выходной карты признаков.

<center><img src="2.1.3.png"></center>
<center>Рис. 3. Схема одного слоя сверточной сети: свертка, за которой следует субдискретизация </center>

Полный процесс изображен на рис. 4.

<center><img src="2.1.4.png"></center>
<center>Рис. 4. Принцип действия свертки</center>

Операция MaxPooling2D (выбоа максимального значения из соседних) уменьшает разрешения карты признаков. Операция заключаетя в следующим: из входной карты признаков извлекается окно, и из него выбирается максимальное значение для каждого канала. Это делается для уменьшения количества коэффициентов в карте признаков для обработки, а также внедрения иерархий пространственных фильтров путем создания последовательных слоев свертки для просмотра все более крупных окон.

Далее добавляем классификатор поверх сверточной нейронной сети. Сеть передает последний выходной тензор с формой (11, 11, 64) на вход полносвязной классифицирующей сети, стека слоев Dense. Эти классификаторы обрабатывают векторы — одномерные массивы, тогда как текущий выход является трехмерным тензором. Выходы (11, 11, 64) преобразуются в векторы с формой (7744,) перед передачей двум полносвязным слоям Dense. Первый слой состоит из 512 нейронов, с функцией активацией _relu_. Последний слой состоит из 10 нейронов с функцией активации _softmax_, возвращающий массив с 10 оценками вероятностей (в сумме дающих 1). Каждая оценка определяет вероятность принадлежности текущего изображения к одному из 10 классов цифр.

Архитектура сети:

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 7744)              0         
                                                                 
 dense (Dense)               (None, 512)               3965440   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                        

    5. Настройка оптимизатора с выбором функции потерь и метрики качества. Число эпох принять от 30 до 50:

Настроить три параметра для этапа компиляции:

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Функцию потерь определяет, как сеть должна оценивать качество своей работы на обучающих данных. Используем функцию потерь _categorical_crossentropy_. Она определяет расстояние между распределениями вероятностей. В данном случае между распределением вероятности на выходе сети и истинным распределением меток. Минимизируя расстояние между этими двумя распределениями, мы учим сеть выводить результат, максимально близкий к истинным меткам.

Оптимизатор — механизм, с помощью которого сеть будет обновлять себя, опираясь на наблюдаемые данные и функцию потерь. Настраиваем модель оптимизатором _rmsprop_ и функцией потерь _mеап squared error_.

Среднеквадратичное распространение корня (_rmsprop_) - это экспоненциально затухающее среднее значение. Существенным свойством _rmsprop_ является то, что вы не ограничены только суммой прошлых градиентов, но вы более ограничены градиентами последних временных шагов. В _rmsprop_ мы пытаемся уменьшить вертикальное движение, используя среднее значение, потому что они суммируются приблизительно до 0, принимая среднее значение. _rmsprop_ предоставляет среднее значение для обновления. Формула обновления изображена на рисунке 5

<center><img src="2.1.8.png"></center>
<center>Рис. 5. Формула rmsprop</center>

Метрика для мониторинга на этапах обучения и тестирования выберем точность (доля правильно классифицированных изображений). _Accuracy_ (точность) — это показатель, который описывает общую точность предсказания модели по всем классам.

    6. Проведение проверки решения, выделяя контрольное множество:

Для контроля точности выделим проверочное множество 10000 образцов из обучающего множества данных:

In [10]:
val_images = train_images[:10000]
partial_train_images = train_images[10000:]
val_labels = train_labels[:10000]
partial_train_labels = train_labels[10000:]

Обучим модель 50 эпохах пакетами (batch_size) по 64 образцов:

In [11]:
history = model.fit(partial_train_images, partial_train_labels, epochs=50, batch_size=64, validation_data=(val_images, val_labels))

Epoch 1/50


782/782 [==============================] - 25s 32ms/step - loss: 0.1319 - accuracy: 0.9588 - val_loss: 0.0640 - val_accuracy: 0.9815
Epoch 2/50
782/782 [==============================] - 23s 30ms/step - loss: 0.0366 - accuracy: 0.9887 - val_loss: 0.0495 - val_accuracy: 0.9861
Epoch 3/50
782/782 [==============================] - 22s 29ms/step - loss: 0.0217 - accuracy: 0.9932 - val_loss: 0.0589 - val_accuracy: 0.9857
Epoch 4/50
782/782 [==============================] - 21s 27ms/step - loss: 0.0139 - accuracy: 0.9957 - val_loss: 0.0684 - val_accuracy: 0.9848
Epoch 5/50
782/782 [==============================] - 22s 28ms/step - loss: 0.0098 - accuracy: 0.9969 - val_loss: 0.0561 - val_accuracy: 0.9879
Epoch 6/50
782/782 [==============================] - 22s 28ms/step - loss: 0.0072 - accuracy: 0.9976 - val_loss: 0.0505 - val_accuracy: 0.9897
Epoch 7/50
782/782 [==============================] - 22s 28ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.0581 - val_accurac

    7. Вывод графиков функции потерь и точности:

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)#accuracy loss_values

plt.plot(epochs, loss_values, 'r', label='Training loss')#ro
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

<center><img src="2.1.5.png"></center>
<center>Рис. 6. Потери на этапах обучения и проверки</center>

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'ro', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

<center><img src="2.1.6.png"></center>
<center>Рис. 7. Точность на этапах обучения и проверки</center>


Лучшее значение метрики функции потерь и точности на валидационном множестве в процессе обучения составляет 0.08 и 0.99 соответственно. Как можно видеть по графикам переобучение наступает после 17 эпохе.

    8. Использование обученной сети для предсказания на новых данных:

In [14]:
results = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 5ms/step - loss: 0.0577 - accuracy: 0.9925


Модель показала точность в 99%, с потерями 0.06. Отличная точность. С подобными данными модель хорошо определяет цифры в виде похожих изображений.

Пример работы обученной сети:

In [ ]:
prediction = model.predict(test_images)
fig = plt.figure
plt.imshow(test_images[0], cmap='gray')
plt.show()
print("Predicted:", np.argmax(prediction[0]), "Target:", np.argmax(test_labels[0]))

<center><img src="2.1.7.png"></center>
<center>Рис. 8. Визуализация предсказания обученной модели.</center>

_Predicted: 7 Target: 7_

## Этап 2. Построение бинарного классификатора. Обучение на малом объеме данных

**Целью данного этапа** лабораторной работы является создание бинарного классификатора изображений кошек и собак из набора данных Cats vs. Dogs.

Вариант этапа 2
| № | Размер входного изображения | Количество слоев | Количество карт признаков на слое | Размер ядра свертки | Количество скрытых слоев классификатора (количество нейронов на слое) |
| :-: | :-: | :-: | :-: | :-: | :-: |
| 3 | 150x150 | 3 | 64-128-128 | 3x3 | 1 (1024) |

 Пошаговая реализация поставленной цели включает:

    1. Загрузка набора данных Cats vs. Dogs:
Набор данных Cats vs. Dogs представляет собой 25000 изображений различного размера. По 12500 изображений выделено для каждого класса (кошек и собак).

    2. Разделение данных на обучающий и тестовый наборы:
Загрузка данных осуществляется по ссылке из Kaggle. Данные изначально не разделены.

Загрузка данных осуществляется при помощи подключения к Google Drive, где хранится архив с данными. Затем этот архив распаковывается в рабочую директорию.

In [ ]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d "bhavikjikadara/dog-and-cat-classification-dataset"
! mkdir datasets
! unzip dog-and-cat-classification-dataset.zip -d datasets
# local_zip = '/content/dog-and-cat-classification-dataset.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/')
# zip_ref.close()

Создадим базовый каталог для сохранения выделенного небольшого набора: 

In [ ]:
base_dir = '/CatsDogs'
os.mkdir(base_dir)
#train_dir = os.path.join(base_dir, 'train')
#validation_dir = os.path.join(base_dir, 'validation')

#train_cats_dir = os.path.join(train_dir, 'cats')
#train_dogs_dir = os.path.join(train_dir, 'dogs')

#validation_cats_dir = os.path.join(validation_dir, 'cats')
#validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Запомним пути в каталоги, которые в последующим будем создавать и использовать.

In [ ]:
original_dataset_dir_Cat = "/content/datasets/PetImages/Cat"
original_dataset_dir_Dog = "/content/datasets/PetImages/Dog"
original_dataset_dir = "/content/datasets/PetImages"

In [ ]:
train_dir = os.path.join(base_dir,'train')
validation_dir  = os.path.join(base_dir,'validation')
test_dir  = os.path.join(base_dir,'test')

train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir,'dogs')

test_cats_dir = os.path.join(test_dir,'cats')
test_dogs_dir = os.path.join(test_dir,'dogs')

Создадим пустые каталоги.

In [ ]:
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)
os.mkdir(train_cats_dir)
os.mkdir(train_dogs_dir)
os.mkdir(validation_cats_dir)
os.mkdir(validation_dogs_dir)
os.mkdir(test_cats_dir)
os.mkdir(test_dogs_dir)

Распределим изображения в обучающий, проверочный и контрольный каталоги:

Разделим их на 2000 обучающих, 1000 проверочных и 1000 контрольных изображений.

In [ ]:
cats_list = os.listdir(original_dataset_dir_Cat)
dogs_list = os.listdir(original_dataset_dir_Dog)

fnames = cats_list[:1000]#1000
for fname in fnames:
    src = os.path.join(original_dataset_dir_Cat, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src,dst)

fnames =cats_list[1000:1500] #1000,1500
for fname in fnames:
    src = os.path.join(original_dataset_dir_Cat, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames =cats_list[1500:2000]#1500,2000
for fname in fnames:
    src = os.path.join(original_dataset_dir_Cat, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames =dogs_list[:1000]
for fname in fnames:
    src = os.path.join(original_dataset_dir_Dog, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src,dst)

fnames =dogs_list[1000:1500]
for fname in fnames:
    src = os.path.join(original_dataset_dir_Dog, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

fnames =dogs_list[1500:2000]
for fname in fnames:
    src = os.path.join(original_dataset_dir_Dog, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


    3. Создание генератора изображений заданной размерности

Создадим объект ImageDataGenerator с несколькими параметрами для аугментации данных (rescale, rotation_range, width_shift, height_shift, shear_range, zoom_range, horizontal_flip, fill_mode) и предварительной обработки изображений. ImageDataGenerator поможет модели выявить больше особенностей данных и достичь лучшей степени обобщения, ImageDataGenerator реализует подход создания дополнительных обучающих данных из имеющихся путем трансформации образцов множеством случайных преобразований, дающих правдоподобные изображения.
*   rescale — масштабирование значения с коэффициентом 1/255.
*   rotation_range — величина в градусах (0–180), диапазон, в котором будет осуществляться случайный поворот изображения;
*   width_shift и height_shift — диапазоны (в долях ширины и высоты), в пределах которых изображения смещаются по горизонтали и вертикали соответственно;
*   shear_range — для случайного применения сдвигового (shearing) преобразования;
*   zoom_range — для случайного изменения масштаба внутри изображений;
*   horizontal_flip — для случайного переворачивания половины изображения по горизонтали — подходит в случае отсутствия предположений о горизонтальной асимметрии;
*   fill_mode — стратегия заполнения вновь созданных пикселов, появляющихся после поворота или смещения по горизонтали/вертикали.

Этот объект будет использоваться для генерации обучающих и валидационных данных.

Обучение сверточной нейронной сети с использованием генераторов расширения данных:

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

    4. Подготовку данных для передачи в нейронную сеть:

Получим тренировочный, валидационный, тестовый генераторы данных, а также словарь сопоставления номера класса с текстовой меткой. Так как используется функция потерь binary_crossentropy, метки должны быть бинарными поставим _class_mode='binary'_, согласно, варианту приведём изображения к размеру _150 × 150: target_size=(150, 150)_.

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


    5. Выполнить конструирование сети: создание сверточной основы в соответствии с вариантом, добавить к сверточной основе классификатор заданной архитектуры, вид активационной функции на сверточном слое и в скрытых слоях классификатора выбрать relu:

С помощью функции _Sequential()_ создаём новую модель. Сверточная часть сети состоит из слоев _Conv2D_ и _MaxPooling2D_. На входе поставим размер изображения _150 x 150_. Количество карт признаков на первом слое _Conv2D_ 64, размер ядра свертки 3x3, с функцией активацией _relu_ после каждого слоя _Conv2D_ применяем _MaxPooling2D_, уменьшая размер карты признаков вдвое. На следующих слоях будем применять тот же стек 2 раза, только изменяя количество карт признаков на слоях _Conv2D_ с 64 на 128. Далее добавляем классификатор поверх сверточной нейронной сети. Первый слой состоит из 1024 нейронов, с функцией активацией _relu_. Последний слой состоит из 1 нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

    6. Настройка оптимизатора с выбором функции потерь и метрики качества. Число эпох принять от 30 до 50:

Функция потерь перекрестная энтропия (binary_crossentropy) - определяет меру расстояния между распределениями вероятностей, или в данном случае - между фактическими данными и предсказаниями. Настраиваем модель с оптимизатором rmsprop.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['acc'])

    7. Проведение проверки решения, выделяя контрольное множество:
Обучение модели производилось в течение 50 эпох, данные разбивались на батчи, размером 20.

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)

Epoch 1/50
100/100 [==============================] - 22s 175ms/step - loss: 0.7241 - acc: 0.4925 - val_loss: 0.6926 - val_acc: 0.5250
Epoch 2/50
100/100 [==============================] - 19s 186ms/step - loss: 0.6927 - acc: 0.5405 - val_loss: 0.7009 - val_acc: 0.5080
Epoch 3/50
100/100 [==============================] - 17s 175ms/step - loss: 0.6882 - acc: 0.5650 - val_loss: 0.6529 - val_acc: 0.6390
Epoch 4/50
100/100 [==============================] - 19s 188ms/step - loss: 0.6688 - acc: 0.5810 - val_loss: 0.6389 - val_acc: 0.5990
Epoch 5/50
100/100 [==============================] - 19s 190ms/step - loss: 0.6737 - acc: 0.6045 - val_loss: 0.6073 - val_acc: 0.6790
Epoch 6/50
100/100 [==============================] - 19s 185ms/step - loss: 0.6608 - acc: 0.6225 - val_loss: 0.5818 - val_acc: 0.7100
Epoch 7/50
100/100 [==============================] - 17s 174ms/step - loss: 0.6508 - acc: 0.6155 - val_loss: 0.5662 - val_acc: 0.6920
Epoch 8/50
100/100 [==============================] - 1

    8. Вывод графиков функции потерь и точности:

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<center><img src="2.2.1.png"></center>
<center>Рис. 9. Точность на этапах обучения и проверки</center>

<center><img src="2.2.2.png"></center>
<center>Рис. 10. Потери на этапах обучения и проверки</center>

    9. Использование обученной сети для предсказания на новых данных:

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=9,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)

Epoch 1/9
100/100 [==============================] - 19s 194ms/step - loss: 0.4951 - acc: 0.7715 - val_loss: 0.4594 - val_acc: 0.7970
Epoch 2/9
100/100 [==============================] - 17s 174ms/step - loss: 0.4773 - acc: 0.7705 - val_loss: 0.6351 - val_acc: 0.7160
Epoch 3/9
100/100 [==============================] - 19s 187ms/step - loss: 0.4854 - acc: 0.7640 - val_loss: 0.4871 - val_acc: 0.7830
Epoch 4/9
100/100 [==============================] - 17s 175ms/step - loss: 0.4891 - acc: 0.7675 - val_loss: 0.4502 - val_acc: 0.7790
Epoch 5/9
100/100 [==============================] - 33s 328ms/step - loss: 0.4718 - acc: 0.7750 - val_loss: 0.4978 - val_acc: 0.7730
Epoch 6/9
100/100 [==============================] - 17s 173ms/step - loss: 0.4839 - acc: 0.7710 - val_loss: 0.4722 - val_acc: 0.7950
Epoch 7/9
100/100 [==============================] - 18s 176ms/step - loss: 0.5186 - acc: 0.7605 - val_loss: 0.4468 - val_acc: 0.7990
Epoch 8/9
100/100 [==============================] - 19s 191ms

In [ ]:
results = model.evaluate(test_generator)
results

50/50 [==============================] - 2s 39ms/step - loss: 0.4906 - acc: 0.7730


[0.4905804991722107, 0.7730000019073486]

Модель демонстрирует результаты на тестовых данных: функция потерь составляет 0.49, а метрика - 0,77. Из-за относительной простоты архитектуры нейронной сети значение метрики несколько снижено. Для повышения качества модели рекомендуется усложнить архитектуру, добавив дополнительные сверточные слои, а также увеличить объем данных для обучения.

Пример работы обученной модели:

<center><img src="2.2.3.png"></center>
<center>Рис. 11. Визуализация предсказания обученной модели.</center>

## Этап 3. Использование предварительно обученной сети.

**Целью данного этапа** Целью данного этапа лабораторной работы является использование предварительно обученной на наборе Imagenet нейронной сети VGG16 для увеличения точности классификации изображений из набора данных Cats vs. Dogs.

Вариант этапа 3
| № | Использование обученной сети | Количество эпох обучения |
| :-: | :-: | :-: |
| 3 | Выделение признаков обучающего набора при помощи сверточной основы с последующим их использованием как входов классификатора | 50 |

 Пошаговая реализация поставленной цели включает:

    1. Загрузку набора данных Cats vs. Dogs и сети VGG16 с весовыми коэффициентами, полученными при обучении на наборе данных Imagenet:

VGG16 - это глубокая сверточная модель нейронной сети, используемая для задач классификации изображений. Сеть состоит из 16 слоев искусственных нейронов, каждый из которых работает над постепенной обработкой информации об изображении и повышением точности ее предсказаний. Она характеризуется своей глубиной, состоящей из 16 слоев, включая 13 сверточных слоев и 3 полностью связанных слоя. Вместо большого количества гиперпараметров, VGG16 использует слои свертки с фильтром 3x3 и шагом 1, которые находятся в том же слое заполнения и maxpool с фильтром 2x2 в шаге 2. Он последовательно использует это расположение слоев свертки и максимального пула во всей архитектуре. В итоге у него есть два полностью соединенных слоя, за которыми следует softmax для вывода. В ней около 138 миллионов параметров.

<center><img src="2.3.1.png"></center>

<center>Рис. 12. Общая структура сети</center>

Архитектура:

Архитектура VGG-16 представляет собой глубокую сверточную нейронную сеть (CNN), разработанную для задач классификации изображений.

Конфигурация VGG-16 обычно состоит из 16 слоев, включая 13 сверточных слоев и 3 полностью связанных слоя. Эти слои организованы в блоки, причем каждый блок содержит несколько сверточных слоев, за которыми следует слой с максимальным объединением для понижающей дискретизации.

<center><img src="2.3.2.png"></center>

<center>Рис. 13. Карта архитектуры VGG-16</center>

Архитектура VGG16 на основе рисунка 13:

1. Входной слой:
* Входные размеры: (224, 224, 3)
2. Сверточные слои (64 фильтра, 3 × 3 фильтра, одинаковое заполнение):
* Два последовательных сверточных слоя с 64 фильтрами в каждом и размером фильтра 3 × 3.
* Такое же заполнение применяется для сохранения пространственных размеров.
3. Максимальный уровень объединения (2 × 2, шаг 2):
* Максимальный уровень объединения с размером пула 2 × 2 и шагом 2.
4. Сверточные слои (128 фильтров, 3 × 3 фильтра, одинаковое заполнение):
* Два последовательных сверточных слоя по 128 фильтров в каждом и размер фильтра 3 × 3.
5. Максимальный уровень объединения (2 × 2, шаг 2):
* Максимальный уровень объединения с размером пула 2 × 2 и шагом 2.
6. Сверточные слои (256 фильтров, 3 × 3 фильтра, одинаковое заполнение):
* Два последовательных сверточных слоя с 256 фильтрами в каждом и размером фильтра 3 × 3.
7. Сверточные слои (512 фильтров, 3 × 3 фильтра, одинаковое заполнение):
* Два набора из трех последовательных сверточных слоев с 512 фильтрами в каждом и размером фильтра 3 × 3.
8. Максимальный уровень объединения (2 × 2, шаг 2):
* Максимальный уровень объединения с размером пула 2 × 2 и шагом 2.
Стек сверточных слоев и максимальный пул:
8. Два дополнительных сверточных слоя после предыдущего стека.
* Размер фильтра: 3 × 3.
9. Сплющивание:
* Сведение выходной карты объектов (7x7x512) в вектор размером 25088.
10. Полностью подключенные слои:
* Три полностью соединенных слоя с активацией ReLU.
* Первый слой с размером ввода 25088 и размером вывода 4096.
* Второй слой с размером ввода 4096 и размером вывода 4096.
* Третий уровень с размером входных данных 4096 и размером выходных данных 1000, соответствующий 1000 классам в ILSVRC challenge.
* Активация Softmax применяется к выходу третьего полностью подключенного уровня для классификации.

Эта архитектура соответствует предоставленным спецификациям, включая использование функции активации ReLU и конечного полностью подключенного уровня, выводящего вероятности для 1000 классов с использованием активации softmax.

Ограничения VGG 16:
* Она очень медленно обучается (оригинальная модель VGG обучалась на графическом процессоре Nvidia Titan в течение 2-3 недель).
* Размер обученных весов ImageNet в VGG-16 составляет 528 МБ. Таким образом, он занимает довольно много места на диске и пропускной способности, что делает его неэффективным.
* 138 миллионов параметров приводят к проблеме резкого увеличения градиентов.

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
  include_top=False,
  input_shape=(150, 150, 3))

58889256/58889256 [==============================] - 0s 0us/step


    2. Разделение данных на обучающий и тестовый наборы:
Загрузка данных осуществляется аналогично как у предыдущего этапа:

In [ ]:
cats_list = os.listdir(original_dataset_dir_Cat)
dogs_list = os.listdir(original_dataset_dir_Dog)

fnames = cats_list[:1000]#1000
for fname in fnames:
    src = os.path.join(original_dataset_dir_Cat, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src,dst)

fnames =cats_list[1000:1500] #1000,1500
for fname in fnames:
    src = os.path.join(original_dataset_dir_Cat, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames =cats_list[1500:2000]#1500,2000
for fname in fnames:
    src = os.path.join(original_dataset_dir_Cat, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames =dogs_list[:1000]
for fname in fnames:
    src = os.path.join(original_dataset_dir_Dog, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src,dst)

fnames =dogs_list[1000:1500]
for fname in fnames:
    src = os.path.join(original_dataset_dir_Dog, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

fnames =dogs_list[1500:2000]
for fname in fnames:
    src = os.path.join(original_dataset_dir_Dog, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

    3. Создание генератора изображений заданной размерности:

Эффективным подходом к глубокому обучению на небольших наборах изображений является использование предварительно обученной сети. Предварительно обученная сеть — это сохраненная сеть, прежде обученная на большом наборе данных, обычно в рамках масштабной задачи классификации изображений. Если исходный набор данных достаточно велик и достаточно обобщен, тогда пространственная иерархия признаков, изученных сетью, может эффективно выступать в роли обобщенной модели видимого мира и быть полезной во многих разных задачах распознавания образов, даже если эти новые задачи будут связаны с совершенно иными классами, отличными от классов в оригинальной задаче. В нашем случае мы возьмем за основу сверточную нейронную сеть, обученную на наборе ImageNet (1,4 миллиона изображений, классифицированных на 1000 разных классов). Коллекция ImageNet содержит множество изображений разных
животных, включая разновидности кошек и собак, а значит, можно рассчитывать, что модель, обученная на этой коллекции, прекрасно справится с нашей задачей классификации изображений кошек и собак

Выделение признаков заключается в использовании представлений, изученных предыдущей сетью, для выделения признаков из новых образцов, которые затем пропускаются через новый классификатор, обучаемый с нуля. Процесс выделения
признаков заключается в том, чтобы взять сверточную основу предварительно обученной сети, пропустить через нее новые данные и на основе вывода обучить новый классификатор.

Получим признаки из изображений путем применения функции _extract_features_.

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features(directory, sample_count):
  features = np.zeros(shape=(sample_count, 4, 4, 512))
  labels = np.zeros(shape=(sample_count))
  generator = datagen.flow_from_directory(
    directory,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')
  i = 0
  for inputs_batch, labels_batch in generator:
    features_batch = conv_base.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break
  return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
1/1 [==============================] - 0s 22ms/step
Found 1000 images belonging to 2 classes.
1/1 [==============================] - 0s 21ms/step
Found 1000 images belonging to 2 classes.
1/1 [==============================] - 0s 25ms/step


    4. Подготовка данных для передачи в нейронную сеть:
Объем тренировочного множества составил 2000 экземпляров, а объем валидационного и тренировочного по 1000 экземпляров. Полученные признаки были приобразованы для подачи в модель нейронной сети методом reshape.

In [ ]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

    5. В соответствие с вариантом сконструируйте сеть, используя вариант классификатора (слои расположенные после сверточной основы) из этапа 2:
С помощью функции _Sequential()_ создаём новую модель. Первый слой состоит из 1024 нейронов, с функцией активацией _relu_. Последний слой состоит из 1 нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность. Для преодоления проблемы переобучения, применим метод Dropout.

Прореживание (dropout) — один из наиболее эффективных и распространенных приемов регуляризации для нейронных сетей. Прореживание, которое применяется к слою, заключается в удалении (присваивании нуля) случайно выбираемым признакам на этапе обучения. . Коэффициент прореживания — это доля обнуляемых признаков, в данном случае выбрали 0.5

In [ ]:
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

    6. Настройка оптимизатора с выбором функции потерь и метрики качества, используя количество эпох обучения согласно варианту;
    7. Проведение проверки решения, выделяя контрольное множество:
Обучение модели производилось в течение 50 эпох, данные разбивались на батчи, размером 20. 

In [ ]:
model.compile(optimizer=RMSprop(lr=2e-5),
 loss='binary_crossentropy',
 metrics=['acc'])
history = model.fit(train_features, train_labels,
 epochs=50,
 batch_size=20,
 validation_data=(validation_features, validation_labels))

Epoch 1/50
100/100 [==============================] - 1s 7ms/step - loss: 1.1621 - acc: 0.7355 - val_loss: 1.2101 - val_acc: 0.5850
Epoch 2/50
100/100 [==============================] - 0s 5ms/step - loss: 0.4358 - acc: 0.8220 - val_loss: 0.3276 - val_acc: 0.8580
Epoch 3/50
100/100 [==============================] - 1s 6ms/step - loss: 0.3187 - acc: 0.8730 - val_loss: 0.3372 - val_acc: 0.8660
Epoch 4/50
100/100 [==============================] - 1s 7ms/step - loss: 0.2841 - acc: 0.8910 - val_loss: 0.2649 - val_acc: 0.8960
Epoch 5/50
100/100 [==============================] - 1s 7ms/step - loss: 0.2428 - acc: 0.9065 - val_loss: 0.3407 - val_acc: 0.8720
Epoch 6/50
100/100 [==============================] - 1s 6ms/step - loss: 0.2054 - acc: 0.9225 - val_loss: 0.2906 - val_acc: 0.8970
Epoch 7/50
100/100 [==============================] - 0s 5ms/step - loss: 0.1910 - acc: 0.9240 - val_loss: 0.2908 - val_acc: 0.8900
Epoch 8/50
100/100 [==============================] - 0s 5ms/step - loss: 0.

    8. Вывод графиков функции потерь и точности:

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<center><img src="2.3.3.png"></center>
<center>Рис. 14. Точность на этапах обучения и проверки</center>

<center><img src="2.3.4.png"></center>
<center>Рис. 15. Потери на этапах обучения и проверки</center>

На 9 эпохе происходит переобучение модели. Лучшее значение функции потерь и метрики на валидационном наборе данных составляют 0.3275 и 0.8880 соответственно.

    9. Использование обученной сети для предсказания на новых данных:

In [ ]:
model.fit(train_features, train_labels,
 epochs=9,
 batch_size=20,
 validation_data=(validation_features, validation_labels))

Epoch 1/9
100/100 [==============================] - 1s 6ms/step - loss: 0.0073 - acc: 0.9970 - val_loss: 1.0577 - val_acc: 0.8750
Epoch 2/9
100/100 [==============================] - 1s 5ms/step - loss: 0.0109 - acc: 0.9970 - val_loss: 1.1070 - val_acc: 0.8880
Epoch 3/9
100/100 [==============================] - 0s 5ms/step - loss: 0.0041 - acc: 0.9975 - val_loss: 1.1580 - val_acc: 0.8850
Epoch 4/9
100/100 [==============================] - 1s 5ms/step - loss: 0.0186 - acc: 0.9935 - val_loss: 1.1561 - val_acc: 0.8970
Epoch 5/9
100/100 [==============================] - 1s 5ms/step - loss: 0.0231 - acc: 0.9920 - val_loss: 1.5697 - val_acc: 0.8820
Epoch 6/9
100/100 [==============================] - 0s 5ms/step - loss: 0.0198 - acc: 0.9955 - val_loss: 1.1466 - val_acc: 0.8940
Epoch 7/9
100/100 [==============================] - 0s 5ms/step - loss: 0.0195 - acc: 0.9930 - val_loss: 1.2472 - val_acc: 0.8810
Epoch 8/9
100/100 [==============================] - 0s 5ms/step - loss: 0.0068 - a

In [ ]:
results = model.evaluate(test_features,test_labels)
results

32/32 [==============================] - 0s 3ms/step - loss: 1.1114 - acc: 0.9010


[1.1113535165786743, 0.9010000228881836]

Итоговое значение функции потерь и метрики на тестовых данных составляет 1.11 и 0.9 соответственно. Для улучшения качества работы модели можно использовать больший объем данных для обучения.

Пример работы обученной модели:

<center><img src="2.3.5.png"></center>
<center>Рис. 16. Визуализация предсказания обученной модели</center>

<p style="text-align: center;">Заключение</p>

1. На первом этапе мы классифицировали изображения цифр из набора данных MNIST. Подготовили данные для передачи в сеть нормализовали входные изображения, и преобразовали метки классов к векторному представлению. Сконструировали сверточную нейронную сеть (CNN).  Сверточная часть сети состоит из слоев _Conv2D_ и _MaxPooling2D_. На входе размер изображения _150 x 150_. Количество карт признаков на первом слое _Conv2D_ 32, размер ядра свертки 3x3, с функцией активацией _relu_ после каждого слоя _Conv2D_ применяем _MaxPooling2D_, уменьшая размер карты признаков вдвое. На следующем слое применили тот же стек, только изменяя количество карт признаков на слоях _Conv2D_ с 32 на 64. Добавили классификатор поверх сверточной нейронной сети. Первый слой состоит из 512 нейронов, с функцией активацией _relu_. Последний слой состоит из 10 нейронов с функцией активации _softmax_, возвращающий массив с 10 оценками вероятностей (в сумме дающих 1). Каждая оценка определяет вероятность принадлежности текущего изображения к одному из 10 классов цифр. Обученная модель обученная модель показала точность около 99%, с потерями 0.06. Наилучшее значение метрики было достигнуто на 17 эпохе и на валидационных данных показала точность в 99%, с потерями 0.08.

2. На втором этапе мы классифицировали изображений кошек и собак из набора данных Cats vs. Dogs. Создали генератор ImageDataGenerator с несколькими параметрами для аугментации данных и предварительной обработки изображений. ImageDataGenerator реализует подход создания дополнительных обучающих данных из имеющихся путем трансформации образцов множеством случайных преобразований, дающих правдоподобные изображения. Сконструировали сверточную нейронную сеть (CNN).  Сверточная часть сети состоит из слоев _Conv2D_ и _MaxPooling2D_. На входе размер изображения _150 x 150_. Количество карт признаков на первом слое _Conv2D_ 64, размер ядра свертки 3x3, с функцией активацией _relu_ после каждого слоя _Conv2D_ применяем _MaxPooling2D_, уменьшая размер карты признаков вдвое. На следующих слоях применили тот же стек 2 раза, только изменяя количество карт признаков на слоях _Conv2D_ с 64 на 128. Добавили классификатор поверх сверточной нейронной сети. Первый слой состоит из 1024 нейронов, с функцией активацией _relu_. Последний слой состоит из 1 нейрона с функцией активации _sigmoid_, на выходе получая скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность. Обученная модель достигла точности около 83%. Снижение точности связано с неоптимальной архитектурой нейронной сети и небольшим количеством обучающих данных. Наилучшее значение метрики было достигнуто на 9 эпохе, и обученная модель показала точность около 90%, с потерями 1.1. 

3. В третьем этапе была таже задача, что и на втором этапе, но был использован метод выделения признаков с помощью сверточной нейронной сети VGG-16. Полученные данные затем подавались на многослойную полносвязную нейронную сеть с двумя слоями. Первый слой состоит из 1024 нейронов, с функцией активацией _relu_. Последний слой состоит из 1 нейрона с функцией активации _sigmoid_, на выходе получаем скалярное значение в диапозоне между 0 и 1, представляющее собой вероятность. Для преодоления проблемы переобучения, как один из наиболее эффективных и распространенных приемов регуляризации для нейронных сетей, применили метод Dropout, с коэффициентом прореживания 0.5. Наилучшее значение метрики было достигнуто на 9 эпохе, и обученная модель показала точность около 77%, с потерями 0.49.

<p style="text-align: center;">Список использованной литературы</p>

1. Шолле Франсуа. Глубокое обучение на Python. - СПб.: Питер, 2018. - 400 с.: ил. - (Серия «Библиотека программиста»).
2. Николенко С., Кадурин А., Архангельская Е. Глубокое обучение. — СПб.: Питер, 2018. — 480 с.: ил. — (Серия «Библиотека программиста»).